In [1]:
#Author: Aisling Scott
#Cleaning data 
#Inputs: 
    #data-merged.csv // Raw data
#Outputs: 
    #vote.csv // Subset of cleaned data
    #nonvote.csv
    #senders.csv
    #returners.csv

In [8]:
#Import libraries needed
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt; plt.rcdefaults()
import numpy as np

# Make sure that graphics appear inline in the iPython notebook
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [9]:
#Read in raw data CSV 
rawdata = pd.read_csv("data-merged.csv")

In [10]:
#rename columns 
rawdata.columns = [col.lower().replace('.', '_') for col in rawdata.columns]

In [12]:
pd.set_option('display.max_columns', None) #set max columns to
#Look at columns 
rawdata.head(5)

,participant_id_in_session,participant_code,participant_label,participant__is_bot,participant__index_in_pages,participant__max_page_index,participant__current_app_name,participant__round_number,participant__current_page_name,participant_ip_address,participant_time_started,participant_exclude_from_data_analysis,participant_visited,participant_mturk_worker_id,participant_mturk_assignment_id,participant_payoff,participant_payoff_plus_participation_fee,session_code,session_label,session_experimenter_name,session_time_scheduled,session_time_started,session_mturk_hitid,session_mturk_hitgroupid,session_comment,session_is_demo,session_config_auto_trust_score,session_config_trust_score,session_config_participation_fee,pre_survey_1_player_id_in_group,pre_survey_1_player_ps_score_value,pre_survey_1_player_ps_score,pre_survey_1_player_computer,pre_survey_1_player_people_best_intentions,pre_survey_1_player_will_to_return_a_favor,pre_survey_1_player_give_to_without_expecting_anything_in_return,pre_survey_1_player_donate,pre_survey_1_player_present,pre_survey_1_player_payoff,pre_survey_1_group_id_in_subsession,pre_survey_1_subsession_round_number,pre_survey_1_subsession_pss_score_median,trust_1_player_id_in_group,trust_1_player_tunderstanding_1,trust_1_player_tunderstanding_2,trust_1_player_tunderstanding_3a,trust_1_player_tunderstanding_3b,trust_1_player_tunderstanding_percentage_1,trust_1_player_tunderstanding_percentage_2,trust_1_player_tunderstanding_percentage_3,trust_1_player_expect_other_player_to_return,trust_1_player_expect_other_player_to_return_revealed,trust_1_player_selected_round_for_payoff,trust_1_player_vote_sequential_sender_returner,trust_1_player_vote_sequential_returner_sender,trust_1_player_vote_simultaneous_sender_returner,trust_1_player_vote_simultaneous_returner_sender,trust_1_player_payoff,trust_1_group_id_in_subsession,trust_1_group_group_play_type,trust_1_group_selected_vote_of_participant_id,trust_1_group_second_vote,trust_1_group_ammount_given,trust_1_group_percentage_sent_back,trust_1_group_ammount_sent_back,trust_1_subsession_round_number,trust_1_subsession_voted_round,trust_1_subsession_reveal_variation,trust_1_subsession_order_variation,trust_1_subsession_play_variation,trust_1_subsession_round_play_type,trust_2_player_id_in_group,trust_2_player_tunderstanding_1,trust_2_player_tunderstanding_2,trust_2_player_tunderstanding_3a,trust_2_player_tunderstanding_3b,trust_2_player_tunderstanding_percentage_1,trust_2_player_tunderstanding_percentage_2,trust_2_player_tunderstanding_percentage_3,trust_2_player_expect_other_player_to_return,trust_2_player_expect_other_player_to_return_revealed,trust_2_player_selected_round_for_payoff,trust_2_player_vote_sequential_sender_returner,trust_2_player_vote_sequential_returner_sender,trust_2_player_vote_simultaneous_sender_returner,trust_2_player_vote_simultaneous_returner_sender,trust_2_player_payoff,trust_2_group_id_in_subsession,trust_2_group_group_play_type,trust_2_group_selected_vote_of_participant_id,trust_2_group_second_vote,trust_2_group_ammount_given,trust_2_group_percentage_sent_back,trust_2_group_ammount_sent_back,trust_2_subsession_round_number,trust_2_subsession_voted_round,trust_2_subsession_reveal_variation,trust_2_subsession_order_variation,trust_2_subsession_play_variation,trust_2_subsession_round_play_type,trust_3_player_id_in_group,trust_3_player_tunderstanding_1,trust_3_player_tunderstanding_2,trust_3_player_tunderstanding_3a,trust_3_player_tunderstanding_3b,trust_3_player_tunderstanding_percentage_1,trust_3_player_tunderstanding_percentage_2,trust_3_player_tunderstanding_percentage_3,trust_3_player_expect_other_player_to_return,trust_3_player_expect_other_player_to_return_revealed,trust_3_player_selected_round_for_payoff,trust_3_player_vote_sequential_sender_returner,trust_3_player_vote_sequential_returner_sender,trust_3_player_vote_simultaneous_sender_returner,trust_3_player_vote_simultaneous_returner_sender,trust_3_player_payoff,trust_3_group_id_in_subsess

In [13]:
#Count people in each session
rawdata.groupby(['session_code'])['session_code'].agg(['count'])

,count
session_code,
06pzxgrf,20
09botyf9,14
3lkq4equ,32
5xj6xfd2,4
96fnvtwk,20
bhtsn6sp,26
d34ljbdl,20
dzkpk7qe,16
k90rtisn,40


In [14]:
#Drop Pilot Sessions
rawdata = rawdata[ rawdata.session_code != '5xj6xfd2']
rawdata = rawdata[ rawdata.session_code != '96fnvtwk']
rawdata = rawdata[ rawdata.session_code != 'vzqen83y']

In [15]:
#Look at number in each session again
rawdata.groupby(['session_code'])['session_code'].agg(['count'])

,count
session_code,
06pzxgrf,20
09botyf9,14
3lkq4equ,32
bhtsn6sp,26
d34ljbdl,20
dzkpk7qe,16
k90rtisn,40
keflcr2b,16
n0gq0iba,24


In [16]:
#Check duplicates in a couple of sessions 
rawdata.duplicated(['session_code', 'participant_code']).sum()

54

In [ ]:
#Indeed, there are duplicates 

In [17]:
#Drop duplicates 
rawdata = rawdata.drop_duplicates(['session_code', 'participant_code'])

In [18]:
rawdata.duplicated(['session_code', 'participant_code']).sum() #Double check duplicates

0

In [19]:
#Look at number in each session again
rawdata.groupby(['session_code'])['session_code'].agg(['count'])
#Now I have no duplicates

,count
session_code,
06pzxgrf,20
09botyf9,14
3lkq4equ,16
bhtsn6sp,26
d34ljbdl,20
dzkpk7qe,16
k90rtisn,20
keflcr2b,16
n0gq0iba,24


In [20]:
#Drop variables we don't need
rawdata = rawdata.drop(['participant_label', 'participant__is_bot', 'participant__index_in_pages', 
              'participant__max_page_index',  'participant__current_app_name', 'participant__round_number',
              'participant__current_page_name', 'participant_ip_address', 'participant_time_started', 
              'participant_exclude_from_data_analysis', 'participant_visited', 'participant_mturk_worker_id',
              'participant_mturk_assignment_id', 'session_experimenter_name', 'session_time_scheduled',
              'session_time_started', 'session_mturk_hitid', 'session_mturk_hitgroupid', 'session_comment',
              'session_is_demo', 'session_config_participation_fee'], axis=1)

## Generating Variables

In [21]:
#Generate a prosocial dummy variable
rawdata['prosocial'] = np.where(rawdata['pre_survey_1_player_ps_score']=="Above the median pro-social-score", 1, 0)

In [22]:
#Make the information treatment variable a dummy for whether the player's type was revealed
rawdata['info_treatment'] = np.where(rawdata['trust_1_subsession_reveal_variation']=="reveal", 1, 0)
#Make the information treatment variable a dummy for whether the player's type was revealed
rawdata['sim_first'] = np.where(rawdata['trust_1_subsession_play_variation']=="simultaneous_first", 1, 0)
#Generate the dummy for whether the first 
rawdata['above_first'] = np.where(rawdata['trust_1_subsession_order_variation']=="first_above", 1, 0)
#Generate Gender dummy variable
rawdata['female'] = np.where(rawdata['post_survey_1_player_gender']== "Female", 1, 0)
#Note I could have used pd.get_dummies, but I wanted to name them and not have dummies for both above and below

#### Personality Measures

In [23]:
#generate personality variables for the Big 5 personality metrics 
#Note: scoring based on https://www.ocf.berkeley.edu/~johnlab/pdfs/BFI-10.doc
rawdata['reserved']= 8- rawdata['post_survey_1_player_reserved_quiet']
rawdata['critical']= 8- rawdata['post_survey_1_player_critical_quarrelsome']
rawdata['disorg']= 8- rawdata['post_survey_1_player_disorganized_careless']
rawdata['anx']= 8- rawdata['post_survey_1_player_anxious_easily_upset']
rawdata['conven']= 8- rawdata['post_survey_1_player_conventional_uncreative']

rawdata['extraversion'] = (rawdata['post_survey_1_player_extraverted_enthusiastic']+ rawdata['reserved'])/2
rawdata['agreeable'] = (rawdata['post_survey_1_player_sympathetic_warm']+ rawdata['critical'])/2
rawdata['conscientious'] = (rawdata['disorg']+ rawdata['post_survey_1_player_dependable_self_disciplined'])/2
rawdata['emotional_stable'] = (rawdata['post_survey_1_player_calm_emotionally_stable'] + rawdata['anx'])/2
rawdata['open'] = (rawdata['post_survey_1_player_open_new_speriences'] + rawdata['conven'])/2

## Renaming variables

In [24]:
#Rename Economics Courses variable
rawdata.rename(columns={'post_survey_1_player_economic_courses': 'econ_courses'}, inplace=True)
#Rename Race variables 
rawdata.rename(columns={'post_survey_1_player_white': 'white', 'post_survey_1_player_asian': 'asian',
                       'post_survey_1_player_black_african_american': 'black',
                        'post_survey_1_player_hispanic_latino': 'hispanic'
                       }, inplace=True)

#Prosocial Survey 
rawdata.rename(columns={'pre_survey_1_player_people_best_intentions': 'people_best_intentions',
                        'pre_survey_1_player_will_to_return_a_favor': 'reciprocate_favor',
                        'pre_survey_1_player_give_to_without_expecting_anything_in_return': 'without_expect',
                        'pre_survey_1_player_donate': 'donate',
                        'pre_survey_1_player_present': 'present',
                        'pre_survey_1_player_ps_score': 'prosocial_score'}, inplace=True)
#Understanding of game
rawdata.rename(columns={'trust_1_player_tunderstanding_1': 'understanding1',
                        'trust_1_player_tunderstanding_2': 'understanding2',
                        'trust_1_player_tunderstanding_3a': 'understanding3',
                        'trust_1_player_tunderstanding_3b': 'understanding4',
                        'trust_1_player_tunderstanding_percentage_1': 'understanding5',
                        'trust_1_player_tunderstanding_percentage_2':  'understanding6',
                        'trust_1_player_tunderstanding_percentage_3': 'understanding7'}, inplace=True)

## Keep a subset of the Columns

In [25]:
#Filter Columns I want for analysis
id1 = ['participant_code', 'session_code', 'info_treatment', 'sim_first', 'above_first', 'prosocial' ]
col_1 = [col for col in rawdata if col.endswith('player_expect_other_player_to_return_revealed')]
col_2 = [col for col in rawdata if col.endswith('player_expect_other_player_to_return')]
col_3 = [col for col in rawdata if col.endswith('_group_ammount_given')]
col_4 = [col for col in rawdata if col.endswith('_group_ammount_sent_back')]
player_id = [col for col in rawdata if col.endswith('_player_id_in_group')]
sim = [col for col in rawdata if col.endswith('_group_play_type')]
personal = ['female', 'white', 'hispanic', 'asian', 'black', 'econ_courses',
           'extraversion', 'agreeable', 'conscientious', 'emotional_stable', 'open',
           'people_best_intentions', 'reciprocate_favor','without_expect', 'donate', 'present',
            'prosocial_score']
vote= ['trust_21_player_vote_sequential_returner_sender', 'trust_22_player_vote_sequential_returner_sender', 
       'trust_23_player_vote_sequential_returner_sender', 'trust_24_player_vote_sequential_returner_sender', 
       'trust_25_player_vote_sequential_returner_sender']
#
col_filtered = id1 + col_1 + col_2 + col_3 + col_4 + player_id + sim + personal + vote


In [26]:
#Create a dataframe just of the columns above
df = rawdata[col_filtered]
df.head(0)
df = df.drop(['pre_survey_1_player_id_in_group' , 'post_survey_1_player_id_in_group'], axis=1)

In [28]:
#check length of data frame
len(df)

218

## Rename Columns 

In [ ]:
# I need to rename the columns in order to reshape from wide to long

In [29]:
#Create an empty dictionary 
dic = { }

In [30]:
for x in df.columns: #search through all the columns 
        m = re.match("trust_(\d+)_group_ammount_given", x) #mark m if the column matches the column
        #print(m)
        if m is not None: #if m has a value add that column to the dictionary with the new name of the column
            dic[x] = "trust_%d" % int(m.group(1))
for x in df.columns:
        m = re.match("trust_(\d+)_player_expect_other_player_to_return_revealed", x)
        if m is not None:
            dic[x] = "reveal_%d" % int(m.group(1))
for x in df.columns:
        m = re.match("trust_(\d+)_group_ammount_sent_back", x)
        if m is not None:
            dic[x] = "return_%d" % int(m.group(1))
for x in df.columns:
        m = re.match("trust_(\d+)_player_id_in_group", x)
        if m is not None:
            dic[x] = "id_in_group_%d" % int(m.group(1))
for x in df.columns:
        m = re.match("trust_(\d+)_group_group_play_type", x)
        if m is not None:
            dic[x] = "sim_%d" % int(m.group(1))
for x in df.columns:
        m = re.match("trust_(\d+)_player_vote_sequential_returner_sender", x)
        if m is not None:
            dic[x] = "vote_sequential_returner_sender_%d" % int(m.group(1))
for x in df.columns:
        m = re.match("trust_(\d+)_player_expect_other_player_to_return", x)
        #print(m)
        if m is not None:
            dic[x] = "expect_%d" % int(m.group(1))

In [31]:
#rename the columns based on their dictionary
df.rename(columns=dic, inplace=True)


In [38]:
#rename the columns
df = df.rename(columns={'sim_first': 'simultaneous_first'})

# Reshape Data

In [39]:
#Reshape data from wide to long 
data = pd.wide_to_long(df, 
                       ["trust_", "expect_", "reveal_", "return_", "id_in_group_", "sim_",
                        "vote_sequential_returner_sender_"],
                       i="participant_code", j="round")

In [41]:
#Flatten out the data by resetting the index
data = data.reset_index()

In [43]:
#Summary statistics 
data.describe()

,round,info_treatment,simultaneous_first,above_first,prosocial,female,white,hispanic,asian,black,econ_courses,extraversion,agreeable,conscientious,emotional_stable,open,people_best_intentions,reciprocate_favor,without_expect,donate,present,trust_,expect_,reveal_,return_,id_in_group_,vote_sequential_returner_sender_
count,5450.000000,5450.000000,5450.000000,5450.000000,5450.000000,5450.000000,5450.000000,5450.000000,5450.000000,5450.000000,5450.000000,5450.000000,5450.000000,5450.000000,5450.000000,5450.000000,5450.000000,5450.000000,5450.000000,5450.000000,5450.000000,5450.000000,2180.000000,1080.000000,5450.000000,5450.000000,1090.000000
mean,13.000000,0.495413,0.486239,0.495413,0.500000,0.605505,0.261468,0.155963,0.582569,0.032110,1.353211,4.107798,4.594037,5.103211,4.472477,5.055046,5.559633,8.733945,7.518349,334.114679,3.444954,6.099817,103.962844,96.734259,6.457006,1.500000,1.790826
std,7.211764,0.500025,0.499856,0.500025,0.500046,0.488787,0.439475,0.362854,0.493181,0.176309,2.575527,1.089492,0.823534,0.918619,0.993960,0.770331,2.167221,1.369128,1.937767,357.138526,1.658934,3.378429,56.405737,57.626443,5.820165,0.500046,0.988598
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,2.500000,2.000000,2.000000,2.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000
25%,7.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.500000,4.000000,4.500000,4.000000,4.500000,4.000000,8.000000,6.000000,100.000000,2.000000,4.000000,62.000000,50.000000,0.330000,1.000000,1.000000
50%,13.000000,0.000000,0.000000,0.000000,0.500000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,4.000000,4.500000,5.500000,4.500000,5.000000,6.000000,9.000000,8.000000,200.000000,4.000000,6.000000,100.000000,100.000000,5.000000,1.500000,1.000000
75%,19.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,2.000000,5.000000,5.000000,6.000000,5.000000,5.500000,7.000000,10.000000,9.000000,500.000000,5.000000,10.000000,150.000000,150.000000,10.500000,2.000000,2.000000
max,25.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,16.000000,6.000000,6.000000,6.000000,6.000000,6.000000,10.000000,10.000000,10.000000,1600.000000,6.000000,10.000000,300.000000,300.000000,30.000000,2.000000,4.000000


In [44]:
#Distinct Counts of People in Each Group 
data.groupby('info_treatment').participant_code.nunique()

info_treatment
0    110
1    108
Name: participant_code, dtype: int64

In [45]:
#Look at means by treatment
data.groupby(['info_treatment'])['trust_'].agg(['mean'])

,mean
info_treatment,
0,6.497455
1,5.694815


## Generate Treatment Variables

In [47]:
#Generate a dummy variable to show if the prosocial partner was the sender 
def f(x):
    if x['id_in_group_'] == 1 and x['prosocial'] == 1 : return 1
    elif x['id_in_group_'] == 1 and x['prosocial'] == 0 : return 0
    elif x['id_in_group_'] == 2 and x['prosocial'] == 0: return 1
    elif x['id_in_group_'] == 2 and x['prosocial'] == 1: return 0

data['pro_first'] = data.apply(f, axis=1)

In [48]:
data.groupby(['id_in_group_', 'prosocial'])['id_in_group_'].agg(['count'])

count
id_in_group_ prosocial       
1            0           1344
             1           1381
2            0           1381
             1           1344

In [49]:
#Generate Treatment Groups  
def f(x):
    if   x['pro_first'] == 1 and x['info_treatment'] == 0 and x['sim_'] == "simultaneous": return 1
    elif x['pro_first'] == 0 and x['info_treatment'] == 0 and x['sim_'] == "simultaneous": return 2
    elif x['pro_first'] == 1 and x['info_treatment'] == 0 and x['sim_'] == "sequential": return 3
    elif x['pro_first'] == 0 and x['info_treatment'] == 0 and x['sim_'] == "sequential": return 4
    elif x['pro_first'] == 1 and x['info_treatment'] == 1 and x['sim_'] == "simultaneous": return 5
    elif x['pro_first'] == 0 and x['info_treatment'] == 1 and x['sim_'] == "simultaneous": return 6
    elif x['pro_first'] == 1 and x['info_treatment'] == 1 and x['sim_'] == "sequential": return 7
    elif x['pro_first'] == 0 and x['info_treatment'] == 1 and x['sim_'] == "sequential": return 8
    else: return 0

data['treatment'] = data.apply(f, axis=1)

In [50]:
data.groupby(['pro_first', 'info_treatment'])['pro_first'].agg(['count'])

count
pro_first info_treatment       
0         0                1332
          1                1356
1         0                1418
          1                1344

## Replace Sender's and returner's data for when they aren't acting

In [51]:
#Generate amount given and amount returned to store the actions their partners did
replace = (data.id_in_group_ == 2)
data.loc[replace ,'amount_given'] = data['trust_']
replace2 = (data.id_in_group_ == 1)
data.loc[replace2 ,'amount_return'] = data['return_']


In [52]:
#Replace trust and return with missing if they aren't the one doing the action 
replace = (data.id_in_group_ == 2)
data.loc[replace ,'trust_']= np.nan
replace2 = (data.id_in_group_ == 1)
data.loc[replace2 ,'return_']= np.nan
#If they are the returner and they don't recieve any trust they don't get a choice to send anything back
#Thus we change their return to zero
replace3 = (data.amount_given == 0.0)
data.loc[replace3, 'return_'] = np.nan

###  Generate Percent Returned 

In [53]:
data['percent_return'] = data['return_']/ (3*data['amount_given'])

In [54]:
#Replace percent_return with zero if return is zero
replace4 = (data.return_ == 0.0) 
data.loc[replace4, 'percent_return'] = 0


In [55]:
#data['percent_return']

In [56]:
data['percent_return_tosender'] = data['amount_return']/ (3*data['trust_'])
replace4 = (data.amount_return == 0.0) 
data.loc[replace4, 'percent_return_tosender'] = 0

In [57]:
data.head()

,participant_code,round,session_code,info_treatment,simultaneous_first,above_first,prosocial,female,white,hispanic,asian,black,econ_courses,extraversion,agreeable,conscientious,emotional_stable,open,people_best_intentions,reciprocate_favor,without_expect,donate,present,prosocial_score,trust_,expect_,reveal_,return_,id_in_group_,sim_,vote_sequential_returner_sender_,pro_first,treatment,amount_given,amount_return,percent_return,percent_return_tosender
0,8as6dcfj,1,09botyf9,1,0,1,1,1,1.0,1.0,0.0,0.0,0.0,4.5,4.0,5.5,3.5,5.0,7.0,10.0,9.0,600.0,4.0,Above the median pro-social-score,5.0,50.0,50.0,NaN,1,sequential,NaN,1,7,NaN,7.0,NaN,0.466667
1,19a9vufr,1,09botyf9,1,0,1,1,1,1.0,1.0,0.0,0.0,1.0,4.0,5.5,5.5,3.0,6.0,4.0,10.0,8.0,600.0,4.0,Above the median pro-social-score,10.0,50.0,24.0,NaN,1,sequential,NaN,1,7,NaN,10.0,NaN,0.333333
2,fzl70u0i,1,09botyf9,1,0,1,1,0,0.0,0.0,1.0,0.0,0.0,4.0,5.0,6.0,5.5,5.5,5.0,10.0,8.0,1500.0,6.0,Above the median pro-social-score,3.0,150.0,100.0,NaN,1,sequential,NaN,1,7,NaN,0.0,NaN,0.000000
3,orxpzwso,1,09botyf9,1,0,1,0,0,0.0,0.0,1.0,0.0,1.0,3.5,3.0,5.0,4.5,5.5,3.0,8.0,3.0,0.0,2.0,Below the median pro-social-score,NaN,NaN,NaN,0.0,2,sequential,NaN,1,7,3.0,NaN,0.0,NaN
4,7gp1fjhy,1,09botyf9,1,0,1,1,1,0.0,0.0,1.0,0.0,0.0,2.5,5.0,6.0,4.5,3.5,5.0,10.0,7.0,600.0,4.0,Above the median pro-social-score,1.0,25.0,0.0,NaN,1,sequential,NaN,1,7,NaN,0.0,NaN,0.000000


In [59]:
#Check data types
data.dtypes
#Notes: Now trust_ and return_ are objects 

participant_code                     object
round                                 int64
session_code                         object
info_treatment                        int64
simultaneous_first                    int64
above_first                           int64
prosocial                             int64
female                                int64
white                               float64
hispanic                            float64
asian                               float64
black                               float64
econ_courses                        float64
extraversion                        float64
agreeable                           float64
conscientious                       float64
emotional_stable                    float64
open                                float64
people_best_intentions              float64
reciprocate_favor                   float64
without_expect                      float64
donate                              float64
present                         

In [60]:
#Verify number of observations
len(data)

5450

In [61]:
data['percent_return'].describe()

count    2461.000000
mean        0.326979
std         0.207065
min         0.000000
25%         0.166667
50%         0.333333
75%         0.500000
max         1.000000
Name: percent_return, dtype: float64

In [62]:
dummies = pd.get_dummies(data['sim_'])
data = pd.concat([data, dummies], axis=1)
dummies = pd.get_dummies(data['round']) #Get Round Dummies
dic = {} #Rename Round Dummies
for x in dummies.columns:
    dic[x] = "round_%d" % int(x)
#rename the columns based on their dictionary
dummies.rename(columns=dic, inplace=True)
data = pd.concat([data, dummies], axis=1)

In [64]:
#create expectations
data['expectations'] = np.where(data['info_treatment']==1, data['reveal_'], data['expect_'])

# Save Data

In [65]:
#Save all data 
data.to_csv('clean-data.csv')

# Create dataframes to Save

### Non voting rounds 

In [66]:
#get non-voting rounds
nonvote = data[data['round'] <=20] 
nonvote.groupby(['treatment'])['treatment'].agg(['count'])

,count
treatment,
1,550
2,550
3,550
4,550
5,540
6,540
7,540
8,540


In [67]:
nonvote.to_csv('nonvote.csv')

### Voting Rounds

In [68]:
vote = data[data['round'] >20] 
vote.to_csv('vote.csv')

### Senders Data

In [69]:
#Make a senders data frame
replace2 = (nonvote.id_in_group_ == 1)
senders = nonvote.loc[replace2]

In [70]:
senders.describe()

,round,info_treatment,simultaneous_first,above_first,prosocial,female,white,hispanic,asian,black,econ_courses,extraversion,agreeable,conscientious,emotional_stable,open,people_best_intentions,reciprocate_favor,without_expect,donate,present,trust_,expect_,reveal_,return_,id_in_group_,vote_sequential_returner_sender_,pro_first,treatment,amount_given,amount_return,percent_return,percent_return_tosender,sequential,simultaneous,round_1,round_2,round_3,round_4,round_5,round_6,round_7,round_8,round_9,round_10,round_11,round_12,round_13,round_14,round_15,round_16,round_17,round_18,round_19,round_20,round_21,round_22,round_23,round_24,round_25,expectations
count,2180.000000,2180.000000,2180.000000,2180.000000,2180.000000,2180.000000,2180.000000,2180.000000,2180.000000,2180.000000,2180.000000,2180.000000,2180.000000,2180.000000,2180.000000,2180.000000,2180.000000,2180.000000,2180.000000,2180.000000,2180.000000,2180.000000,2180.000000,1080.000000,0.0,2180.0,0.0,2180.000000,2180.000000,0.0,2180.000000,0.0,2180.000000,2180.000000,2180.000000,2180.000000,2180.000000,2180.000000,2180.000000,2180.000000,2180.000000,2180.000000,2180.000000,2180.000000,2180.000000,2180.000000,2180.000000,2180.000000,2180.000000,2180.000000,2180.000000,2180.000000,2180.000000,2180.000000,2180.000000,2180.0,2180.0,2180.0,2180.0,2180.0,2180.000000
mean,10.500000,0.495413,0.486239,0.495413,0.500000,0.605505,0.261468,0.155963,0.582569,0.032110,1.353211,4.107798,4.594037,5.103211,4.472477,5.055046,5.559633,8.733945,7.518349,334.114679,3.444954,6.091284,103.962844,96.734259,NaN,1.0,NaN,0.500000,4.481651,NaN,6.496005,NaN,0.301077,0.500000,0.500000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.0,0.0,0.0,0.0,0.0,102.369266
std,5.767604,0.500094,0.499925,0.500094,0.500115,0.488854,0.439535,0.362904,0.493248,0.176333,2.575882,1.089642,0.823647,0.918745,0.994097,0.770437,2.167519,1.369317,1.938033,357.187693,1.659162,3.303581,56.405737,57.626443,NaN,0.0,NaN,0.500115,2.291740,NaN,5.761289,NaN,0.218368,0.500115,0.500115,0.217995,0.217995,0.217995,0.217995,0.217995,0.217995,0.217995,0.217995,0.217995,0.217995,0.217995,0.217995,0.217995,0.217995,0.217995,0.217995,0.217995,0.217995,0.217995,0.217995,0.0,0.0,0.0,0.0,0.0,58.198753
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,2.500000,2.000000,2.000000,2.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,1.0,NaN,0.000000,1.000000,NaN,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
25%,5.750000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.500000,4.000000,4.500000,4.000000,4.500000,4.000000,8.000000,6.000000,100.000000,2.000000,4.000000,62.000000,50.000000,NaN,1.0,NaN,0.000000,2.000000,NaN,1.000000,NaN,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,50.000000
50%,10.500000,0.000000,0.000000,0.000000,0.500000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,4.000000,4.500000,5.500000,4.500000,5.000000,6.000000,9.000000,8.000000,200.000000,4.000000,6.000000,100.000000,100.000000,NaN,1.0,NaN,0.500000,4.000000,NaN,5.000000,NaN,0.333333,0.500000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,100.000000
75%,15.250000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,2.000000,5.000000,5.000000,6.0

In [71]:
#Get trust summary stats to make sure data is right 
trust = senders.groupby(['treatment'])['trust_'].agg(['mean', 'count', 'std'])
trust

,mean,count,std
treatment,,,
1,6.367273,275,2.974285
2,5.843636,275,3.419713
3,7.072727,275,3.056968
4,6.705455,275,3.373978
5,4.900000,270,3.183543
6,5.937037,270,3.228549
7,5.270370,270,3.269359
8,6.603704,270,3.349858


In [72]:
senders.to_csv('senders.csv')

In [73]:
#print(senders[senders['treatment']==1])

## Returner Data Frame

In [75]:
#Make a senders data frame
replace2 = (nonvote.id_in_group_ == 2)
returners = nonvote.loc[replace2]
#returners

In [76]:
returned = returners.groupby(['treatment'])['return_'].agg(['mean', 'std'])
returned

,mean,std
treatment,,
1,6.922874,5.681215
2,7.509600,5.890551
3,8.354717,6.510090
4,8.832669,5.894734
5,4.707076,4.182925
6,6.553893,5.083164
7,5.775862,4.876103
8,8.180328,5.391681


In [ ]:
## Write returner's data to csv

In [77]:
returners.to_csv('returners.csv')